In [4]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("gpt-5")
standard_model = init_chat_model("gpt-5-nano")


@wrap_model_call
def state_based_model(request: ModelRequest, 
handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """Select model based on State conversation length."""
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)  

    if message_count > 10:
        # Long conversation - use model with larger context window
        model = large_model
    else:
        # Short conversation - use efficient model
        model = standard_model

    request = request.override(model=model)  

    return handler(request)

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    middleware=[state_based_model],
    system_prompt="You are roleplaying a real life helpful office intern."
)

In [7]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?")
        ]}
)

print(response["messages"][-1].content)

I didn’t water it today—I don’t have access to the office to do it physically. But I can help you set things up so it doesn’t get missed.

Want me to:
- Kick off a recurring reminder (e.g., every Friday at 3 PM) to water plant X?
- Create a simple care log (date watered, plant name, notes) in a shared sheet?
- Draft a quick Slack/Teams message to the team asking someone to water it on a set day?

If you share the plant name and your preferred schedule, I’ll tailor it and set it up. Quick care tip: water when the top inch of soil is dry, and ensure the pot drains well.


In [8]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-nano-2025-08-07


In [9]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When should we expect to have to replace the pot?")
        ]}
)

print(response["messages"][-1].content)

Generally every 12–18 months for faster growers, or 2–3 years for slow ones. Best time is early spring.

Let’s plan to check the roots now:
- If they’re circling the surface or coming out the drainage holes, or the soil dries out very fast, we’ll repot in March.
- If not, we can just refresh the top inch of soil now and aim to repot late spring/summer.

When we do, we’ll move up just 1–2 inches in pot diameter, use fresh potting mix, and pause fertilizer for about 4–6 weeks after repotting.


In [10]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-2025-08-07
